<a href="https://colab.research.google.com/github/Tharindu021/Context_Based_Spell_Checker/blob/main/SpellChecker1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
from nltk.corpus import wordnet
from nltk.metrics.distance import edit_distance
import pandas as pd
import numpy as np
import gensim
from gensim.models import Word2Vec
import string
from nltk.corpus import stopwords
from transformers import BertTokenizer, BertModel , AutoTokenizer, AutoModel, AutoModelForMaskedLM
import torch
import csv
import random

**connect to google drice**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**get the pretrained model path, load the models and get the tokenizer.**

In [ ]:
from gensim.models import KeyedVectors
model_path = "/content/drive/MyDrive/Dataset/Models/google_news.kv"
model = KeyedVectors.load(model_path)
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

**load the dictionary and the dataset containing misspelled words with their correct word mappings**

In [ ]:
path = "/content/drive/MyDrive/Dataset/english Dictionary only words.csv"

df = pd.read_csv(path)
dictionary = set(df['word'].str.lower())

def load_dataset(dataset_path):
    dataset = {}
    try:
        with open(dataset_path, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            if 'Incorrect Word' not in reader.fieldnames or 'Correct Word' not in reader.fieldnames:
                raise ValueError("CSV file must have 'Incorrect Word' and 'Correct Word' columns.")

            for row in reader:
                incorrect_word = row['Incorrect Word'].strip().lower()
                correct_word = row['Correct Word'].strip()
                dataset[incorrect_word] = correct_word
    except FileNotFoundError:
        print(f"Error: File not found at {dataset_path}")
    except Exception as e:
        print(f"An error occurred: {e}")
    return dataset

dataset_path = "/content/drive/MyDrive/Dataset/research_dataset.csv"
dataset = load_dataset(dataset_path)
print(dict(list(dataset.items())[-5:]))

# def load_testing_dataset(testing_dataset_path):
#     with open(testing_dataset_path, 'r') as file:
#         reader = csv.DictReader(file)
#         testing_dataset = [row for row in reader]
#     return testing_dataset

# testing_dataset_path = "/content/drive/MyDrive/Dataset/mispelld sentence and correct sentences/dataset_with_misspelled_words.csv"
# testing_dataset = load_testing_dataset(testing_dataset_path)
# print(len(testing_dataset))



<ipython-input-5-606d292397b4>:3: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


An error occurred: 'NoneType' object has no attribute 'strip'
{'eyar': 'year', 'yera': 'year', 'eyasr': 'years', 'yeasr': 'years', 'yersa': 'years'}


**get punkt to preprocess sentences**

In [ ]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

**to get the similarity between two vectors**

In [ ]:
def compute_similarity(vector1, vector2):
    return np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))

**in this function i get the how much matchable my target word to the sentence**

In [ ]:
def get_original_word_confidence(masked_sentence, original_word, tokenizer, model):
    # get the bert numbers for the masked sentence words
    inputs = tokenizer(masked_sentence, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)

    # get the mask token index
    mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
    # get the mask token prediction tokens
    mask_token_logits = outputs.logits[0, mask_token_index, :]

    # Get the score for the original word at the mask position
    original_token_id = tokenizer.convert_tokens_to_ids(original_word)
    # check the original word contain in the mask_token_logits and get the probobility for it using softmax
    original_token_score = torch.softmax(mask_token_logits, dim=1)[0, original_token_id].item()

    return original_token_score

**in this function, I get the best suggested words for my masked word to replace it.**

In [ ]:
def get_best_replacement(masked_sentence, tokenizer, model):
    # initialize the exclude tokens most of the time model recomend thos puncutations so i removed them
    exclude_tokens = [".", ",", ";", ":", "?", "!"]
    # get the corresponds number in bert
    inputs = tokenizer(masked_sentence, return_tensors="pt")

    # turn of the gradient calculation and produce set of logits for each recomended word
    with torch.no_grad():
        outputs = model(**inputs)

    # get the mask word index in input ids, [1] we are using 2d array so using 1 we can get the index [[1,2,3,103]]
    mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]

    # and then get the relevant row using mask token index for predictions
    mask_token_logits = outputs.logits[0, mask_token_index, :]

    #convert logits to probabilities
    probabilities = torch.softmax(mask_token_logits, dim=1)

    #get the top 10 predictions
    top_tokens = torch.topk(probabilities, k=10, dim=1)

    # for token_id in top_tokens.indices[0]:
    #    print(f"Best replacements words :'{tokenizer.decode([token_id.item()])}'")

    #loop each top predictions with pair of id and score
    for token_id, score in zip(top_tokens.indices[0], top_tokens.values[0]):
      #get the word for token id
      replacement = tokenizer.decode([token_id.item()]).strip()

      #if we found replacement that not in exclude tokens and also it is valid word then we can return it
      if replacement not in exclude_tokens and replacement.isalpha():
          return replacement, score.item()

    # if no valid words found then it return  first of the probability
    top_token = top_tokens.indices[0, 0].item()
    top_score = top_tokens.values[0, 0].item()
    replacement = tokenizer.decode([top_token]).strip()
    return replacement, top_score

I am using downloaded pretained model useto the checking the spell checking and also following function is to find most similar_word in given input words to the sentence, Firstly it tokenise the sentence and then check the words are in the model if not it return none and next get the sentence mean base on the word it store in the sentence_mean then it measure the cosine similarity between sentence and relevant word and it label to the word then its store in the simillarities variable then sort it in descending order return the most 5 similar word.

In [ ]:
def find_most_similar_word(sentence, input_words, word2vec_model):
    # tokenise the sentence words
    sentence_words = [word for word in sentence if word in word2vec_model]
    #if there is not words then return it
    if not sentence_words:
        return None

    # get the whole sentence mean vector from getting each embeded word mean
    sentence_mean = np.mean([word2vec_model[word] for word in sentence_words], axis=0)

    # get the word2vec model have words
    valid_words = [word for word in input_words if word in word2vec_model]

    #if there is not words then return it
    if not valid_words:
        return None

    # each words array words get as the label and compoute the similarity between sentence and each word and store it as label and similarity
    similarities = [(label, compute_similarity(sentence_mean, word2vec_model[label])) for label in valid_words]

    # get highest similarity as first and return similarity words
    similarities.sort(key=lambda x: x[1], reverse=True)
    most_similar_words = [label for label, _ in similarities[:5]]
    return most_similar_words

get the simillar words using bert model

In [ ]:
def find_most_similar_word_using_bert(sentence, input_words, bert_model):
    tokenizer = BertTokenizer.from_pretrained(bert_model)
    model = AutoModel.from_pretrained(bert_model)
    #set model evalution mode
    model.eval()

    with torch.no_grad():
        # tokenize the sentence and get as tensor
        sentence_tokens = tokenizer(sentence, return_tensors='pt')
        # get the sentene embedding as 1d vector
        sentence_embedding = model(**sentence_tokens).last_hidden_state.mean(dim=1).squeeze()

    most_similar_word = None
    # initialise to negative infinty
    highest_similarity = -float('inf')

    for word in input_words:
        if not isinstance(word, str):
              word = str(word)

        with torch.no_grad():
            # tokenise word return as tensor
            word_tokens = tokenizer(word, return_tensors='pt')
            # get the word embedding as 1d vector
            word_embedding = model(**word_tokens).last_hidden_state.mean(dim=1).squeeze()

        # compute cosine similarity between sentence and word and convert tensors to numpy array
        similarity = torch.cosine_similarity(sentence_embedding, word_embedding, dim=0).item()
        #torch.nn.functional.cosine_similarity(sentence_embedding, word_embedding, dim=0)

        # update the similar word
        if similarity > highest_similarity:
            highest_similarity = similarity
            most_similar_word = word

    return most_similar_word

check sentence exist real word errors,if there has real word errors then give the correct sentence

In [ ]:
def real_word_errors_check(sentence, bert_model_name="bert-base-uncased"):
    # load the pretained tokenizer and model
    tokenizer = BertTokenizer.from_pretrained(bert_model_name)
    model = AutoModelForMaskedLM.from_pretrained(bert_model_name)

    # get the sentence word
    words = sentence.split()
    # initialize the dictionary for set the word confidence for sentence
    word_confidences = {}

    print("Checking for potential real-word errors...\n")

    # get each of word in words array and calculate confidence using mask the word
    for i, word in enumerate(words):

        masked_sentence = sentence.replace(word, "[MASK]", 1)
        confidence = get_original_word_confidence(masked_sentence, word, tokenizer, model)

        # save confidence
        word_confidences[word] = confidence

    # display confidence for each word
    print("Word Confidence Scores:")
    for word, confidence in word_confidences.items():
        print(f"{word}: {confidence}")

    # ge the minimum confidence score word as target_word
    target_word = min(word_confidences, key=word_confidences.get)

    # using this function we can get the best replacement for target word
    replacement, _ = get_best_replacement(sentence.replace(target_word, "[MASK]", 1), tokenizer, model)

    # check replacment word is same as the target word, if the two words are not the same, replace the target word with the replacement.
    if replacement.lower() != target_word.lower():
        corrected_sentence = sentence.replace(target_word, replacement, 1)
        print(f"Suggested replacement for '{target_word}': '{replacement}'")
    else:
        corrected_sentence = sentence
        print("No corrections needed.")

    print("\Recomended Sentence:", corrected_sentence)
    return corrected_sentence

**In this function we use the edit distance algorithm to calculate the minimum distance between a correct word and a misspelled word.i updated the algorithm to check for character swaps, making it more efficient in calculating the distance.**

In [ ]:
def edit_distance1(s1, s2):
    m, n = len(s1), len(s2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    # Initialize the first row and column
    for i in range(m + 1):
        dp[i][0] = i
    for j in range(n + 1):
        dp[0][j] = j

    # Fill in the DP table
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if s1[i - 1] == s2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = min(dp[i - 1][j - 1] + 1,  # Substitution
                               dp[i - 1][j] + 1,      # Deletion
                               dp[i][j - 1] + 1)      # Insertion

                # Check for transposition
                if i > 1 and j > 1 and s1[i - 1] == s2[j - 2] and s1[i - 2] == s2[j - 1]:
                    dp[i][j] = min(dp[i][j], dp[i - 2][j - 2] + 1)

    return dp[m][n]


**In this function, we retrieve up to 30 similar words using the edit distance algorithm.**

In [ ]:
def recomend_simillar_words(words):
  # initialize the variables
  simillar_words = []

  # get the each word from the dictionary and then calculate the edit_distance value if the distance value is lower than 2 then it push to the similar_words array
  for dict_words in dictionary:
    distance = edit_distance1(str(dict_words), str(words))
    if distance < 2:
      simillar_words.append( (dict_words , distance) )
  # sort the similar words using distance ascending order
  simillar_words.sort(key=lambda x : x[1])
  # return 30 tuples
  return [ word[0] for word in simillar_words[:10]]

**This spell checker function retrieves similar words using the edit distance algorithm, processes them through additional functions to identify the most similar word, and finally calculates accuracy metrics**

In [ ]:
def spell_checker(sentence, dictionary, dataset):

    # initialize the variables
    similar_correct = 0
    similar_total = 0
    recommended_correct = 0
    recommended_total = 0
    similar_words = []
    correct_word = None

    # remove punctuation marks
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))
    # tokenize the input sentence
    words = nltk.word_tokenize(sentence)

    bert_model_name  = 'bert-base-uncased'

    # get each tokenize word from words
    for word in words:

        if word.lower() not in dictionary:

            # get the correct word from the dataset if there is not exist word then correct_word going to be none
            correct_word = dataset.get(word.lower(), None)
            print(f"\nCorect Word for '{word}' : {correct_word}")

            # get the similar words to the choosen words from edit distance algorithm
            similar_words = recomend_simillar_words(word.lower())
            print(f"\nFirst dictionary check for similar words to '{word}': {similar_words}")

            # count the similar words total and correct similar word
            if similar_words:
                similar_total += len(similar_words)
                if correct_word in similar_words:
                    similar_correct += 1

            # if the length of similar words is 1, we can assume that the one similar word is the correct word
            if(len(similar_words) == 1):
                print(f"Similar words for '{word}' are: {similar_words}")
            else :
                # if similar_words contains more than one word, we first check these words using the Word2Vec model to get the most similar word among them.
                recommended_words = find_most_similar_word(sentence, similar_words, model)
                print(f"Recommended words for '{word}' are: {recommended_words}")

                # count the recommend words total and recommnend correct word
                if recommended_words:
                    recommended_total += len(recommended_words)
                    if correct_word in recommended_words:
                        recommended_correct += 1

                # secondly check the words using bert modal
                recommended_word_from_bert = find_most_similar_word_using_bert(sentence, similar_words, bert_model_name)
                print(f"Recommended word using bert model '{word}': {recommended_word_from_bert}")

                # count the recommend words total and recommnend correct word
                if recommended_word_from_bert:
                    recommended_total += len(recommended_word_from_bert)
                    if correct_word in recommended_word_from_bert:
                        recommended_correct += 1


    # check sentence exist real word errors and recomend a correct sentence using bert modal
    real_word_errors_check(sentence,bert_model_name)

**In that part of the code, we can input a sentence, and it checks whether the sentence contains any misspelled words. If there is any misspelled word, it recommends similar words to user**

In [ ]:
user_input = ""
while user_input.lower() != "exit":
    user_input = input("Please enter the sentence you want or type 'exit' to quit: ")
    if user_input.lower() == "exit":
        print("Exiting...")
    else:
        spell_checker(user_input, dictionary, dataset)

Please enter the sentence you want or type 'exit' to quit: i eat appel

Corect Word for 'appel' : apple

First dictionary check for similar words to 'appel': ['apple', 'appeal', 'rappel']
Recommended words for 'appel' are: ['apple', 'appeal', 'rappel']
Recommended word using bert model 'appel': apple


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Checking for potential real-word errors...

Word Confidence Scores:
i: 0.09052897989749908
eat: 0.0025490790139883757
appel: 4.195733982470529e-09
Best replacements words :'.'
Best replacements words :';'
Best replacements words :'!'
Best replacements words :'?'
Best replacements words :'...'
Best replacements words :','
Best replacements words :'|'
Best replacements words :'and'
Best replacements words :'।'
Best replacements words :'-'
Suggested replacement for 'appel': 'and'
\Recomended Sentence: i eat and
Please enter the sentence you want or type 'exit' to quit: exit
Exiting...


**Try to meassure accuracy**